In [1]:
from kalmanfilter import KalmanFilter
from datapoint import DataPoint
from fusionekf import FusionEKF
from tools import get_RMSE
import numpy as np

In [2]:
def parse_data(file_path):
  """
    Args:
    file_path 
      - path to a text file with all data. 
      - each line should have the following format:
          [SENSOR ID] [SENSOR RAW VALUES] [TIMESTAMP] [GROUND TRUTH VALUES]

          Example 1: 
          line with three measurements from a radar sensor in polar coordinate 
          followed by a timestamp in unix time 
          followed by the the "ground truth" which is
          actual real position and velocity in cartesian coordinates (four state variables)

          R 8.46642 0.0287602 -3.04035  1477010443399637  8.6 0.25  -3.00029  0
          (R) (rho) (phi) (drho) (timestamp) (real x) (real y) (real vx) (real vy)

          Example 2:
          line with two measurements from a lidar sensor in cartesian coordinates 
          followed by a timestamp in unix time
          followed by the the "ground truth" which is 
          the actual real position and velocity in cartesian coordinates (four state variables)

          L 8.44818 0.251553  1477010443449633  8.45  0.25  -3.00027  0

    Returns:
      all_sensor_data, all_ground_truths
      - two lists of DataPoint() instances 

  """

  all_sensor_data = []
  all_ground_truths = []  

  with open(file_path) as f:
      
    for line in f:
      data = line.split() 
      
      if data[0]  == 'L':
        
        sensor_data = DataPoint({ 
          'timestamp': int(data[3]),
          'name': 'lidar',
          'x': float(data[1]), 
          'y': float(data[2])
        })
        
        g = {'timestamp': int(data[3]),
             'name': 'state',
             'x': float(data[4]),
             'y': float(data[5]),
             'vx': float(data[6]),
             'vy': float(data[7])
        }
          
        ground_truth = DataPoint(g)
                
      elif data[0] == 'R':
          
        sensor_data = DataPoint({ 
          'timestamp': int(data[4]),
          'name': 'radar',
          'rho': float(data[1]), 
          'phi': float(data[2]),
          'drho': float(data[3])
        })
       
        g = {'timestamp': int(data[4]),
             'name': 'state',
             'x': float(data[5]),
             'y': float(data[6]),
             'vx': float(data[7]),
             'vy': float(data[8])
        }
        ground_truth = DataPoint(g)  

      all_sensor_data.append(sensor_data)
      all_ground_truths.append(ground_truth)

  return all_sensor_data, all_ground_truths

In [3]:
def get_state_estimations(EKF, all_sensor_data):
  """
  Calculates all state estimations given a FusionEKF instance() and sensor measurements

  Args:
    EKF - an instance of a FusionEKF() class 
    all_sensor_data - a list of sensor measurements as a DataPoint() instance

  Returns:
    all_state_estimations 
      - a list of all state estimations as predicted by the EKF instance
      - each state estimation is wrapped in  DataPoint() instance
  """

  all_state_estimations = []

  for data in all_sensor_data:
    
    EKF.process(data)
  
    x = EKF.get()
    px, py, vx, vy = x[0, 0], x[1, 0], x[2, 0], x[3, 0]

    g = {'timestamp': data.get_timestamp(),
         'name': 'state',
         'x': px,
         'y': py,
         'vx': vx,
         'vy': vy }

    state_estimation = DataPoint(g)  
    all_state_estimations.append(state_estimation)
    
  return all_state_estimations 

In [4]:
def print_EKF_data(all_sensor_data, all_ground_truths, all_state_estimations, RMSE):
  """
  Prints all relevant EKF data in a nice formal

  Args:
    all_sensor_data
      - a list of sensor measurements as DataPoint() instances
    all_state_estimations
      - a list of state estimations as DataPoint() instances
    all_ground_truths
      - a list of ground truths as DataPoint instances
    RMSE
      - a list of the four computed root-mean-square error of the four state variables considered

    Returns: None 
  """

  px, py, vx, vy = RMSE 

  print("-----------------------------------------------------------")
  print('{:10s} | {:8.3f} | {:8.3f} | {:8.3f} | {:8.3f} |'.format("RMSE:", px, py, vx, vy))
  print("-----------------------------------------------------------")
  print("NUMBER OF DATA POINTS:", len(all_sensor_data))
  print("-----------------------------------------------------------")  

  i = 1
  for s, p, t in zip(all_sensor_data, all_state_estimations, all_ground_truths):
      
    print("-----------------------------------------------------------")
    print("#", i, ":", s.get_timestamp())
    print("-----------------------------------------------------------")  

    if s.get_name() == 'lidar':
      x, y = s.get_raw()
      print('{:15s} | {:8.3f} | {:8.3f} |'.format("LIDAR:", x, y))
    else:
      rho, phi, drho = s.get_raw()
      print('{:15s} | {:8.3f} | {:8.3f} | {:8.3f} |'.format("RADAR:", rho, phi, drho))  

    x, y, vx, vy = p.get()
    print('{:15s} | {:8.3f} | {:8.3f} | {:8.3f} | {:8.3f} |'.format("PREDICTION:", x, y, x, y))  

    x, y, vx, vy = t.get()
    print('{:15s} | {:8.3f} | {:8.3f} | {:8.3f} | {:8.3f} |'.format("TRUTH:", x, y, x, y))  

    i += 1

In [5]:
lidar_R = np.matrix([[0.01, 0], 
                     [0, 0.01]])

radar_R = np.matrix([[0.01, 0, 0], 
                     [0, 1.0e-6, 0], 
                     [0, 0, 0.01]])

lidar_H = np.matrix([[1, 0, 0, 0],
                     [0, 1, 0, 0]])

P = np.matrix([[1, 0, 0, 0],
               [0, 1, 0, 0],
               [0, 0, 1000, 0], 
               [0, 0, 0, 1000]])

Q = np.matrix(np.zeros([4, 4]))
F = np.matrix(np.eye(4))

d = {
  'number_of_states': 4, 
  'initial_process_matrix': P,
  'radar_covariance_matrix': radar_R,
  'lidar_covariance_matrix': lidar_R, 
  'lidar_transition_matrix': lidar_H,
  'inital_state_transition_matrix': F,
  'initial_noise_matrix': Q, 
  'acceleration_noise_x': 5, 
  'acceleration_noise_y': 5
}

EKF1 = FusionEKF(d)
EKF2 = FusionEKF(d)

In [6]:
all_sensor_data, all_ground_truths = parse_data("data/data-1.txt")
all_state_estimations = get_state_estimations(EKF1, all_sensor_data)
px, py, vx, vy = get_RMSE(all_state_estimations, all_ground_truths)

print_EKF_data(all_sensor_data, all_ground_truths, all_state_estimations, 
               RMSE = [px, py, vx, vy])

-----------------------------------------------------------
RMSE:      |    0.025 |    0.023 |    0.339 |    0.370 |
-----------------------------------------------------------
NUMBER OF DATA POINTS: 1224
-----------------------------------------------------------
-----------------------------------------------------------
# 1 : 1477010443399637
-----------------------------------------------------------
RADAR:          |    8.466 |    0.029 |   -3.040 |
PREDICTION:     |    8.463 |    0.243 |    8.463 |    0.243 |
TRUTH:          |    8.600 |    0.250 |    8.600 |    0.250 |
-----------------------------------------------------------
# 2 : 1477010443449633
-----------------------------------------------------------
LIDAR:          |    8.448 |    0.252 |
PREDICTION:     |    8.448 |    0.252 |    8.448 |    0.252 |
TRUTH:          |    8.450 |    0.250 |    8.450 |    0.250 |
-----------------------------------------------------------
# 3 : 1477010443499690
---------------------------

In [7]:
all_sensor_data, all_ground_truths = parse_data("data/data-2.txt")
all_state_estimations = get_state_estimations(EKF2, all_sensor_data)
px, py, vx, vy = get_RMSE(all_state_estimations, all_ground_truths)

print_EKF_data(all_sensor_data, all_ground_truths, all_state_estimations, 
               RMSE = [px, py, vx, vy])

-----------------------------------------------------------
RMSE:      |    0.174 |    0.165 |    0.394 |    0.706 |
-----------------------------------------------------------
NUMBER OF DATA POINTS: 200
-----------------------------------------------------------
-----------------------------------------------------------
# 1 : 1477010443349642
-----------------------------------------------------------
LIDAR:          |    0.000 |    0.000 |
PREDICTION:     |    0.000 |    0.000 |    0.000 |    0.000 |
TRUTH:          |    0.000 |    0.000 |    0.000 |    0.000 |
-----------------------------------------------------------
# 2 : 1477010443349642
-----------------------------------------------------------
RADAR:          |    0.000 |    0.000 |    0.000 |
PREDICTION:     |    0.000 |    0.000 |    0.000 |    0.000 |
TRUTH:          |    0.000 |    0.000 |    0.000 |    0.000 |
-----------------------------------------------------------
# 3 : 1477010444349642
----------------------------